In [1]:
import os
%load_ext autoreload
%autoreload 2

from FlexivPy.joy import XBoxController
joy = XBoxController(0)

pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: XDG_RUNTIME_DIR is invalid or not set in the environment.


Put your stick at reset and do not touch it while calibrating


In [2]:
joy.getStates()

{'left_joy': array([ 0., -0.]),
 'right_joy': array([ 0., -0.]),
 'left_trigger': 0.0004999999999999449,
 'right_trigger': 0.0004999999999999449,
 'A': 0,
 'B': 0,
 'X': 0,
 'Y': 0,
 'left_bumper': 0,
 'right_bumper': 0,
 'options_left': 0,
 'options_right': 0,
 'left_joy_btn': 0,
 'right_joy_btn': 0}

Before going through the next cells, start the asynchronous simulator or the real robot bridge by tunning the following in your terminal:

**Asynchronous Simulator:**

```bash
flexivpy_async_sim --mode velocity
```

**Real Robot Client**

```bash
robot_server -cm 3 -g --path /home/FlexivPy/FlexivPy/assets/ -rcf /home/FlexivPy/flexivpy_bridge/config.yaml
```
The `cm` 3 means that the robot is started in joint velocity mode.

In [3]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
model = FlexivModel()
robot =FlexivDDSClient()

ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred


waiting to receive the first message from the robot...
Robot is ready!


In [ ]:
from FlexivPy.robot import FlexivCmd
import time
for i in range(200):
    cmd = FlexivCmd()
    cmd.dq = np.array([-0.0, 0.00, 0, 0, 0, 0, 0])
    cmd.mode=3
    robot.set_cmd(cmd)
    time.sleep(0.01)

In [4]:
from FlexivPy.controllers.jointspace import RRTController
from FlexivPy.controllers.runners import blocking_runner
homing_controller = RRTController(goal_tolerance=0.005)
blocking_runner(robot, homing_controller)

goal reached


<ControllerStatus.GOAL_REACHED: 1>

In [5]:
from FlexivPy.controllers.runners import NonBlockingRunner
from FlexivPy.controllers.taskspace import DiffIKController
task_controller = DiffIKController(model, dt=0.01, dq_max=1., control_mode='velocity')

In [6]:
runner = NonBlockingRunner(robot, task_controller, timeout=120.)

ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred


max time reached


In [7]:
import time
import numpy as np
import datetime
time.sleep(0.2)
state = robot.get_robot_state()

if state is not None:
    print('starting the controller')
    info = model.getInfo(np.array(state.q), np.array(state.dq))
    T0 = info['poses']['link7']
    # Initialize the desired pose
    x0, y0, z0 = 0.0, 0.0, 0.0 
    R0 = np.eye(3)
    start_time = time.time()
    while time.time()-start_time < 100:
        joy_state = joy.getStates()
        left_joy = joy_state['left_joy']
        right_joy = joy_state['right_joy']
        
        if joy_state['right_bumper']==0:
            vx_cmd = right_joy[1]
            vy_cmd = right_joy[0]
            vz_cmd = left_joy[0]
            if np.abs(vx_cmd)<0.1:
                vx_cmd=0
            if np.abs(vy_cmd)<0.1:
                vy_cmd=0
            if np.abs(vz_cmd)<0.1:
                vz_cmd=0
            y0 = y0 + vy_cmd*0.2/100
            x0 = x0 + vx_cmd*0.2/100
            z0 = z0 - vz_cmd*0.2/100
        else:
            wx_cmd = right_joy[1]
            wy_cmd = right_joy[0]
            wz_cmd = left_joy[0]
            if np.abs(wx_cmd)<0.1:
                wx_cmd=0
            if np.abs(wy_cmd)<0.1:
                wy_cmd=0
            if np.abs(wz_cmd)<0.1:
                wz_cmd=0
            cmd = np.array([wx_cmd,wy_cmd,wz_cmd])
            omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                                [ cmd[2],   0,      -cmd[0]],
                                [-cmd[1],   cmd[0],      0]])
            R0 = R0@(np.eye(3)+omega_hat/100)

        time.sleep(0.01)
        T_cmd= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
        task_controller.T_cmd = T_cmd
        # state = robot.get_robot_state()
        # info = model.getInfo(np.array(state.q), np.array(state.dq))
        # J = info['Js']['link7']
        # print(np.linalg.det(J@J.T))
    print('Demo ended.')
else:
    print('State is None. check the connection')

starting the controller


ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8750:(snd_pcm_recover) underrun occurred


KeyboardInterrupt: 